In [2]:
# Cell 1: Enhanced Setup and Imports
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import enhanced modules
from src.data.dataset import ChestXrayDataset, create_data_splits, calculate_class_weights
from src.data.dataloader import create_rtx3060_dataloaders
from src.models.enhanced_efficientnet_b4 import (
    create_enhanced_efficientnet_b4,
    create_enhanced_focal_loss
)
from src.trainer.enhanced_trainer import EnhancedTrainer

print("✅ Enhanced modules imported successfully")
print("🎯 Target: 0.85+ AUC with advanced features")
print("🔥 Features: Multi-attention, Pyramid pooling, TTA, Mixup/Cutmix")

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(f"\n🎉 CLARITy Enhanced EfficientNet-B4 Training")
print(f"   From: Fast Stable B3 (0.7118 AUC)")
print(f"   To: Enhanced B4 (0.85+ AUC target)")

✅ Enhanced modules imported successfully
🎯 Target: 0.85+ AUC with advanced features
🔥 Features: Multi-attention, Pyramid pooling, TTA, Mixup/Cutmix
🚀 Using device: cuda
   GPU: NVIDIA GeForce RTX 3060
   VRAM: 12.9 GB

🎉 CLARITy Enhanced EfficientNet-B4 Training
   From: Fast Stable B3 (0.7118 AUC)
   To: Enhanced B4 (0.85+ AUC target)


In [3]:
# Cell 2: Load Data Pipeline for Enhanced Training
BASE_PATH = Path(r"D:/Projects/CLARITY/Model/Dataset/archive")  # Update this path!

print("🔄 Loading data pipeline for enhanced training...")

# Load metadata
data_entry_path = BASE_PATH / "Data_Entry_2017.csv"
df = pd.read_csv(data_entry_path)
print(f"✅ Metadata loaded: {len(df):,} entries")

# Create image mapping
image_mapping = {}
for main_folder in sorted(BASE_PATH.iterdir()):
    if main_folder.is_dir() and main_folder.name.startswith('images_'):
        images_subfolder = main_folder / 'images'
        if images_subfolder.exists():
            for img_file in images_subfolder.glob("*.png"):
                image_name = img_file.name
                if image_name not in image_mapping:
                    image_mapping[image_name] = img_file

print(f"✅ Image mapping: {len(image_mapping):,} images")

# Create data splits
train_df, val_df, test_df = create_data_splits(df, 
                                               test_size=0.15,
                                               val_size=0.10,
                                               random_seed=42)

print(f"✅ Data splits: Train({len(train_df):,}) Val({len(val_df):,}) Test({len(test_df):,})")

# Display data distribution
print(f"\n📊 Dataset Statistics:")
print(f"   Total images: {len(df):,}")
print(f"   Training set: {len(train_df):,} ({len(train_df)/len(df)*100:.1f}%)")
print(f"   Validation set: {len(val_df):,} ({len(val_df)/len(df)*100:.1f}%)")
print(f"   Test set: {len(test_df):,} ({len(test_df)/len(df)*100:.1f}%)")

🔄 Loading data pipeline for enhanced training...
✅ Metadata loaded: 112,120 entries
✅ Image mapping: 112,120 images
Patient-level data splits:
  Train: 83,847 images from 23,105 patients (74.8%)
  Val:   11,550 images from 3,080 patients (10.3%)
  Test:  16,723 images from 4,620 patients (14.9%)
✅ No patient overlap verified - clean splits!
✅ Data splits: Train(83,847) Val(11,550) Test(16,723)

📊 Dataset Statistics:
   Total images: 112,120
   Training set: 83,847 (74.8%)
   Validation set: 11,550 (10.3%)
   Test set: 16,723 (14.9%)


In [4]:
# Cell 3: Create Enhanced Datasets
print("🔄 Creating enhanced datasets...")

# ENHANCED SETTINGS - Optimized for maximum performance
ENHANCED_IMAGE_SIZE = 448    # Higher resolution for B4
ENHANCED_BATCH_SIZE = 6      # Smaller batch due to model complexity
ENHANCED_WORKERS = 4         # Optimized workers
ENHANCED_ACCUMULATION = 6    # Effective batch = 36

print(f"⚙️  Enhanced Training Configuration:")
print(f"   Image size: {ENHANCED_IMAGE_SIZE}×{ENHANCED_IMAGE_SIZE}")
print(f"   Batch size: {ENHANCED_BATCH_SIZE}")
print(f"   Effective batch: {ENHANCED_BATCH_SIZE * ENHANCED_ACCUMULATION}")
print(f"   Workers: {ENHANCED_WORKERS}")
print(f"   Target: 0.85+ AUC")
print(f"   Expected time per epoch: 20-25 minutes")
print(f"   Expected total time: 20-25 hours")

# Create enhanced datasets with maximum augmentation
train_dataset = ChestXrayDataset(
    df=train_df,
    image_mapping=image_mapping,
    image_size=ENHANCED_IMAGE_SIZE,
    is_training=True,
    augmentation_prob=0.95  # Maximum augmentation for generalization
)

val_dataset = ChestXrayDataset(
    df=val_df,
    image_mapping=image_mapping,
    image_size=ENHANCED_IMAGE_SIZE,
    is_training=False
)

test_dataset = ChestXrayDataset(
    df=test_df,
    image_mapping=image_mapping,
    image_size=ENHANCED_IMAGE_SIZE,
    is_training=False
)

print(f"✅ Enhanced datasets created!")
print(f"   Training samples: {len(train_dataset):,}")
print(f"   Validation samples: {len(val_dataset):,}")
print(f"   Test samples: {len(test_dataset):,}")

🔄 Creating enhanced datasets...
⚙️  Enhanced Training Configuration:
   Image size: 448×448
   Batch size: 6
   Effective batch: 36
   Workers: 4
   Target: 0.85+ AUC
   Expected time per epoch: 20-25 minutes
   Expected total time: 20-25 hours
Dataset created with 83847 samples
Training mode: True
Image size: 448x448

Label matrix created: (83847, 15)
Positive samples per class:
  No Finding...............  45146
  Atelectasis..............   8720
  Cardiomegaly.............   2019
  Effusion.................  10071
  Infiltration.............  14772
  Mass.....................   4477
  Nodule...................   4691
  Pneumonia................   1062
  Pneumothorax.............   3981
  Consolidation............   3458
  Edema....................   1738
  Emphysema................   1794
  Fibrosis.................   1236
  Pleural_Thickening.......   2562
  Hernia...................    171
Transforms created for training
Dataset created with 11550 samples
Training mode: False
Imag

In [5]:
# Cell 4: Create Enhanced DataLoaders
print("🔄 Creating enhanced dataloaders...")

# Calculate enhanced class weights with minority boosting
class_weights = calculate_class_weights(train_dataset.labels, method='inverse_freq_sqrt')

# Apply additional minority class boosting
minority_indices = [-1, -2, -3]  # Hernia, Pleural_Thickening, Fibrosis
original_weights = class_weights.clone()

print("🔍 Class weight analysis:")
disease_classes = [
    'No Finding', 'Atelectasis', 'Cardiomegaly', 'Effusion', 
    'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 
    'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 
    'Pleural_Thickening', 'Hernia'
]

for i, disease in enumerate(disease_classes):
    count = np.sum(train_dataset.labels[:, i])
    weight = class_weights[i].item()
    print(f"   {disease}: {count:,} samples, weight: {weight:.3f}")

# Boost minority classes even more
for idx in minority_indices:
    class_weights[idx] *= 2.0  # Additional 2x boost
    print(f"   🔥 {disease_classes[idx]} boosted: {original_weights[idx].item():.3f} → {class_weights[idx].item():.3f}")

print(f"✅ Enhanced class weights calculated with minority boosting")

# Create enhanced dataloaders
train_loader, val_loader, test_loader = create_rtx3060_dataloaders(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    test_dataset=test_dataset,
    batch_size=ENHANCED_BATCH_SIZE,
    num_workers=ENHANCED_WORKERS,
    use_weighted_sampling=True
)

print(f"✅ Enhanced dataloaders created!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")
print(f"   Test batches: {len(test_loader)}")

🔄 Creating enhanced dataloaders...

Class weights (inverse_freq_sqrt):
--------------------------------------------------
No Finding...............    1.000 (pos:  45146)
Atelectasis..............    1.000 (pos:   8720)
Cardiomegaly.............    1.000 (pos:   2019)
Effusion.................    1.000 (pos:  10071)
Infiltration.............    1.000 (pos:  14772)
Mass.....................    1.000 (pos:   4477)
Nodule...................    1.000 (pos:   4691)
Pneumonia................    1.000 (pos:   1062)
Pneumothorax.............    1.000 (pos:   3981)
Consolidation............    1.000 (pos:   3458)
Edema....................    1.000 (pos:   1738)
Emphysema................    1.000 (pos:   1794)
Fibrosis.................    1.000 (pos:   1236)
Pleural_Thickening.......    1.000 (pos:   2562)
Hernia...................    1.000 (pos:    171)
🔍 Class weight analysis:
   No Finding: 45,146.0 samples, weight: 1.000
   Atelectasis: 8,720.0 samples, weight: 1.000
   Cardiomegaly: 2,019.0

In [6]:
# Cell 5: Create Enhanced EfficientNet-B4 Model
print("🔄 Creating enhanced EfficientNet-B4 model...")

# Create the most advanced model
model = create_enhanced_efficientnet_b4(
    num_classes=15,
    class_weights=class_weights
)

# Move to device
model = model.to(device)

# Create enhanced focal loss with strong minority boosting
criterion = create_enhanced_focal_loss(
    class_weights=class_weights,
    minority_boost=5.0  # 5x boost for minority classes
)

# Print model statistics
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Enhanced Model Statistics:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Architecture: Enhanced EfficientNet-B4")
print(f"   Input resolution: {ENHANCED_IMAGE_SIZE}×{ENHANCED_IMAGE_SIZE}")

print(f"\n🔥 Advanced Features:")
print(f"   • Multi-scale attention ✅")
print(f"   • Pyramid pooling ✅")
print(f"   • Self-attention mechanism ✅")
print(f"   • Enhanced focal loss ✅")
print(f"   • Minority class boosting (5x) ✅")
print(f"   • Label smoothing ✅")

# Enable optimizations
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = False

# Memory check
if torch.cuda.is_available():
    print(f"\n💾 Memory Status:")
    print(f"   GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"   GPU memory cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")

print("✅ Enhanced model ready!")

🔄 Creating enhanced EfficientNet-B4 model...
✅ Enhanced EfficientNet-B4 Created:
   Feature dim: 1792
   Advanced attention: ✅
   Pyramid pooling: ✅
   Multi-scale fusion: ✅
   Parameters: 74,286,268
✅ Enhanced Focal Loss Created:
   Alpha: 0.25
   Gamma: 2.5
   Minority boost: 5.0x
   Label smoothing: 0.1

📊 Enhanced Model Statistics:
   Total parameters: 74,286,268
   Trainable parameters: 74,286,268
   Architecture: Enhanced EfficientNet-B4
   Input resolution: 448×448

🔥 Advanced Features:
   • Multi-scale attention ✅
   • Pyramid pooling ✅
   • Self-attention mechanism ✅
   • Enhanced focal loss ✅
   • Minority class boosting (5x) ✅
   • Label smoothing ✅

💾 Memory Status:
   GPU memory allocated: 0.30 GB
   GPU memory cached: 0.31 GB
✅ Enhanced model ready!


In [7]:
# Cell 6: Initialize Enhanced Trainer
print("🔄 Initializing enhanced trainer...")

# Create enhanced trainer with all advanced features
trainer = EnhancedTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    device=device,
    
    # ENHANCED LEARNING SETTINGS
    learning_rate=2e-5,              # Conservative for B4
    weight_decay=1e-4,               # Moderate regularization
    accumulation_steps=ENHANCED_ACCUMULATION,
    mixed_precision=True,            # FP16 for speed
    gradient_clipping=0.5,           # Strong clipping for stability
    
    # ADVANCED FEATURES
    scheduler_type='onecycle',       # Advanced scheduling
    use_mixup_cutmix=True,          # Data augmentation
    test_time_augmentation=True,     # TTA for validation
    minority_class_boost=True,       # Special minority handling
    
    # TRAINING CONFIGURATION
    max_epochs=60,                   # Longer training for excellence
    patience=12,                     # Patient early stopping
    checkpoint_dir='../models/enhanced_checkpoints'
)

print("✅ Enhanced trainer initialized!")
print(f"\n⚙️ Training Configuration:")
print(f"   Max epochs: 60")
print(f"   Learning rate: 2e-5 (conservative)")
print(f"   Scheduler: OneCycleLR (advanced)")
print(f"   Gradient clipping: 0.5 (strong)")
print(f"   Mixed precision: FP16 ✅")
print(f"   Mixup/Cutmix: ✅")
print(f"   Test-time augmentation: ✅")
print(f"   Early stopping patience: 12 epochs")

print(f"\n🎯 Performance Targets:")
print(f"   Primary target: 0.85+ AUC")
print(f"   Secondary target: 0.80+ Macro-F1")
print(f"   Expected improvement: +0.13-0.15 AUC over B3")

print(f"\n⏱️ Time Estimates:")
print(f"   Expected time per epoch: 20-25 minutes")
print(f"   Total expected time: 20-25 hours")
print(f"   Early completion possible: ~12-15 hours if target reached")

🔄 Initializing enhanced trainer...
✅ Enhanced trainer initialized!

⚙️ Training Configuration:
   Max epochs: 60
   Learning rate: 2e-5 (conservative)
   Scheduler: OneCycleLR (advanced)
   Gradient clipping: 0.5 (strong)
   Mixed precision: FP16 ✅
   Mixup/Cutmix: ✅
   Test-time augmentation: ✅
   Early stopping patience: 12 epochs

🎯 Performance Targets:
   Primary target: 0.85+ AUC
   Secondary target: 0.80+ Macro-F1
   Expected improvement: +0.13-0.15 AUC over B3

⏱️ Time Estimates:
   Expected time per epoch: 20-25 minutes
   Total expected time: 20-25 hours
   Early completion possible: ~12-15 hours if target reached


In [8]:
# Cell 7: Load Previous Models (Optional Transfer Learning)
print("🔄 Checking for previous models for transfer learning...")

# Option 1: Load from B3 fast/stable model
try:
    b3_checkpoint_path = "../models/fast_stable_checkpoints/best_fast_stable_model.pth"
    if Path(b3_checkpoint_path).exists():
        print("🔍 Found Fast Stable B3 model - attempting transfer learning...")
        
        b3_checkpoint = torch.load(b3_checkpoint_path, map_location=device, weights_only=False)
        
        # Extract compatible weights
        b3_dict = b3_checkpoint.get('model_state_dict', {})
        model_dict = model.state_dict()
        
        # Filter compatible weights (mainly backbone layers)
        compatible_dict = {}
        incompatible_layers = []
        
        for k, v in b3_dict.items():
            if k in model_dict:
                if v.size() == model_dict[k].size():
                    compatible_dict[k] = v
                else:
                    incompatible_layers.append(k)
            else:
                incompatible_layers.append(k)
        
        if compatible_dict:
            model_dict.update(compatible_dict)
            model.load_state_dict(model_dict)
            
            print(f"✅ Transfer learning applied successfully!")
            print(f"   Compatible layers: {len(compatible_dict)}")
            print(f"   Total model layers: {len(model_dict)}")
            print(f"   Transfer efficiency: {len(compatible_dict)/len(model_dict)*100:.1f}%")
            print(f"   Previous B3 AUC: {b3_checkpoint.get('best_val_auc', 0.7118):.4f}")
            
            if incompatible_layers:
                print(f"   Incompatible layers: {len(incompatible_layers)} (will train from scratch)")
        else:
            print("⚠️ No compatible layers found - starting from ImageNet pretrained")
    else:
        print("⚠️ B3 checkpoint not found - starting from ImageNet pretrained")
        
except Exception as e:
    print(f"⚠️ Transfer learning failed: {e}")
    print("Starting from ImageNet pretrained weights...")

# Option 2: Load from DenseNet121 (if available)
try:
    densenet_path = "../models/saved_models/enhanced_densenet121/enhanced_densenet121.pth"
    if Path(densenet_path).exists():
        print("🔍 DenseNet121 model found but skipping (different architecture)")
    else:
        print("ℹ️ No DenseNet121 model found")
except:
    pass

print("\n✅ Model initialization complete - ready for training!")

🔄 Checking for previous models for transfer learning...
🔍 Found Fast Stable B3 model - attempting transfer learning...
✅ Transfer learning applied successfully!
   Compatible layers: 169
   Total model layers: 788
   Transfer efficiency: 21.4%
   Previous B3 AUC: 0.7118
   Incompatible layers: 423 (will train from scratch)
ℹ️ No DenseNet121 model found

✅ Model initialization complete - ready for training!


In [9]:
# Cell 8: Start Enhanced Training
print("🎯 Starting Enhanced EfficientNet-B4 Training...")
print("   From: Fast Stable B3 (0.7118 AUC)")
print("   To: Enhanced B4 (0.85+ AUC target)")
print("   Features: Multi-attention, TTA, Mixup/Cutmix, Pyramid pooling")
print("   Expected time: 20-25 hours")
print("=" * 100)

import time
start_time = time.time()

# Clear CUDA cache before training
torch.cuda.empty_cache()

# Memory monitoring
if torch.cuda.is_available():
    print(f"🖥️ Pre-training GPU memory:")
    print(f"   Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"   Cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")

print(f"\n🚀 Training started at: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# Run enhanced training
try:
    best_auc, best_f1 = trainer.train_for_excellence()
    training_completed = True
except KeyboardInterrupt:
    print("\n⏸️ Training interrupted by user")
    best_auc = trainer.best_val_auc
    best_f1 = trainer.best_val_f1
    training_completed = False
except Exception as e:
    print(f"\n❌ Training error: {e}")
    best_auc = trainer.best_val_auc
    best_f1 = trainer.best_val_f1
    training_completed = False

training_time = time.time() - start_time

print(f"\n🏆 ENHANCED TRAINING SESSION COMPLETED!")
print(f"=" * 70)
print(f"🎯 Final Results:")
print(f"   Best AUC: {best_auc:.4f}")
print(f"   Best Macro-F1: {best_f1:.4f}")
print(f"   Training time: {training_time/3600:.2f} hours")
print(f"   Completed epochs: {len(trainer.val_aucs) if hasattr(trainer, 'val_aucs') else 'N/A'}")
if hasattr(trainer, 'val_aucs') and trainer.val_aucs:
    print(f"   Average time per epoch: {training_time/len(trainer.val_aucs)/60:.1f} minutes")

# Performance assessment
print(f"\n📊 Performance Assessment:")
if best_auc >= 0.85:
    print(f"✅ EXCELLENCE ACHIEVED! Target 0.85+ AUC reached!")
    print(f"🎉 Ready for clinical deployment consideration")
    assessment = "EXCELLENCE"
elif best_auc >= 0.82:
    print(f"🔥 HIGH PERFORMANCE! Very close to excellence")
    print(f"💡 Consider ensemble or fine-tuning for 0.85+")
    assessment = "HIGH_PERFORMANCE"
elif best_auc >= 0.80:
    print(f"📈 GREAT PROGRESS! Significant improvement")
    print(f"💡 Ready for ensemble combination")
    assessment = "GREAT_PROGRESS"
elif best_auc >= 0.75:
    print(f"📊 GOOD FOUNDATION! Solid improvement achieved")
    print(f"💡 Consider ensemble approach")
    assessment = "GOOD_FOUNDATION"
else:
    print(f"⚠️ NEEDS IMPROVEMENT! Consider longer training or ensemble")
    assessment = "NEEDS_IMPROVEMENT"

# Improvement calculation
if best_auc > 0.7118:  # B3 baseline
    improvement = best_auc - 0.7118
    print(f"\n📈 Improvement over Fast Stable B3:")
    print(f"   AUC improvement: +{improvement:.4f}")
    print(f"   Relative improvement: +{improvement/0.7118*100:.2f}%")

🎯 Starting Enhanced EfficientNet-B4 Training...
   From: Fast Stable B3 (0.7118 AUC)
   To: Enhanced B4 (0.85+ AUC target)
   Features: Multi-attention, TTA, Mixup/Cutmix, Pyramid pooling
   Expected time: 20-25 hours
🖥️ Pre-training GPU memory:
   Allocated: 0.45 GB
   Cached: 0.47 GB

🚀 Training started at: 2025-10-05 15:06:53
🎯 Starting Enhanced Training for Excellence



⏸️ Training interrupted by user

🏆 ENHANCED TRAINING SESSION COMPLETED!
🎯 Final Results:
   Best AUC: 0.0000
   Best Macro-F1: 0.0000
   Training time: 0.11 hours
   Completed epochs: 0

📊 Performance Assessment:
⚠️ NEEDS IMPROVEMENT! Consider longer training or ensemble
